<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/JonasNylund/Lab-2/jonasnylund_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix Factorization**
**Jonas Nylund**

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [0]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Jonas Nylund (jonasnyl@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import numpy as np;
from numpy import random;
from scipy import sparse;
from scipy.sparse import csr_matrix;

import unittest


# **Introduction**

Give a short description of the problem investigated in the report, and provide some background information so that the reader can understand the context. 

Briefly describe what method you have chosen to solve the problem, and justify why you selected that method. 


Efficient matrix algorithms are useful for solving linear algebra problems. Here are some implemented.


# **Methods**

**Sparse Matrix-vector product**

For sparse matrices, where most of the cells are 0, it is very wasteful to do dense matrix multiplication. Instead, for sparse matrices, we can implement "sparse multiplication", where only the cells actually containing data are multiplied with a vector **x**.



In [0]:
def sparseProduct(x, val, col_idx, row_ptr):
  ## Assume that x and Matrix are the same dimensions. Cannot really be 
  ## checked here since the format does not support it.
  assert len(x) >= max(col_idx);


  out = np.zeros(len(row_ptr)-1);
  for i in range(len(row_ptr)-1):
    rs = row_ptr[i];
    re = row_ptr[i+1];
    for j in range(re-rs):
      col = col_idx[rs+j];
      v = val[rs+j];
      out[i] += v*x[col];

  return out;


In [0]:
m = np.random.randint(2,10);
n = np.random.randint(2,10);
A = csr_matrix(sparse.random(m,n,0.25));
x = random.rand(n);

print(m,n);
print(A @ x);
print(sparseProduct(x, A.data, A.indices, A.indptr));


5 7
[0.81479742 0.         0.60141252 1.23667301 0.53705956]
[0.81479742 0.         0.60141252 1.23667301 0.53705956]


**QR factorization**

Factorizing a matrix **A** into a orthogonal matrix **Q** and a triangular matrix **R** can be used to solve certain linear algebra equations efficiently. The factorization can be done in multiple ways, here is an implementation of the modified Gram–Schmidt process.

In [0]:
def qrFactorize(mat):
  A = np.array(mat);
  m = A.shape[0];
  n = A.shape[1];
  assert m >= n;

  #==============================================#
  # https://f.kth.se/sangbok/#!/chapter/8/song/16
  i = 0;

  ##Tag en delrumsbas M och en vektor a
  M = [];
  while i < n:
    a = np.array(A[:,i]);

    ##Projicera ner, tag dess residual
    for j in range(i):
      a -= np.dot(A[:,i], M[j])*M[j];

    ## Normalisera, tillför den till M
    M.append( a/np.linalg.norm(a) );

    ##Ta sen nästa vektor, börja om igen
    i += 1;
  #==============================================#

  Q = np.zeros(A.shape);
  R = np.zeros((n,n));
  for i in range(n):
    Q[:,i] = M[i];
    for j in range(i, n):
      R[i,j] = np.dot(A[:,j], M[i]);

  return Q, R;


In [8]:
n = np.random.randint(2,10);
m = np.random.randint(n,10);

A = np.random.rand(m,n);
Q,R = qrFactorize(A);
q,r = np.linalg.qr(A);

print("R triangular:", np.allclose(R, np.triu(R, 0)));
print("||Q@R - A|| =",np.linalg.norm((Q @ R) - A, ord='fro'));
print("||Q^T@Q - I|| =",np.linalg.norm((Q.transpose() @ Q) - np.eye(n), ord='fro'));
print("Equal to numpy solution:", np.allclose(np.abs(Q),np.abs(q)) and np.allclose(np.abs(R),np.abs(r)));

R triangular: True
||Q@R - A|| = 7.871593303505863e-16
||Q^T@Q - I|| = 4.025725416188366e-15
Equal to numpy solution: True


**Direct solver**

Solving Systems of linear equations are done by matrix inversion. Here, we use QR decomposition to invert a matrix. THe back substitution algorithm uses Gauss elimination. The system could be solved without inverting R, but this was simpler to implement

In [0]:
def backSubst(mat):
  ## A must be square
  A = np.array(mat);
  assert A.shape[0] == A.shape[1];
  
  I = np.eye(A.shape[0]);
  n = A.shape[0];

  for i in range(n):
    I[i,:] /= A[i,i];
    A[i,:] /= A[i,i];
  
  ## Gaussian elimination, backward 
  for i in range(1,n):
    I[n-i,:] /= A[n-i,n-i];
    A[n-i,:] /= A[n-i,n-i];

    for j in range(1, n-i+1):
      f = A[n-i-j,n-i];
      I[n-i-j,:] -= I[n-i,:]*f;
      A[n-i-j,:] -= A[n-i,:]*f;

  return I;

def invert(A):
  assert A.shape[0] == A.shape[1];
  ## Invert A by QR-factorization
  Q,R = qrFactorize(A);
  Qi = Q.transpose(); ## invert Q by transposing
  Ri = backSubst(R);  ## invert R by back substitution
  return (Ri @ Qi);

def solve(A, b):
  return invert(A) @ b;


In [0]:
m = np.random.randint(2,10);

A = np.random.rand(m,m)*1000-500;
b = np.random.rand(m)*1000-500;
x = solve(A,b);
print("||A@x - b|| =", np.linalg.norm(A@x - b));

A = np.random.rand(m,m)*1000-500;
y = np.random.rand(m)*1000-500;
b = A@y;
x = solve(A,b);
print("||x-y|| =",np.linalg.norm(x-y));

||A@x - b|| = 3.9992890150046743e-13
||x-y|| = 7.568029316369878e-10


**Least squares problem**

THe least squares problem is often encountered in science, for fitting data to equations for example. The least squares problem can also be solved with QR decomposition (what a wonderful thing!)

In [0]:
def leastSquare(A, b):
  Q1,R1 = qrFactorize(A);
  Qi = Q1.transpose();
  Ri = backSubst(R1);
  return Ri @ (Qi @ b);

In [0]:
m = np.random.randint(2,10);
n = m + np.random.randint(2,10);

A = np.random.rand(n,m);
b = np.random.rand(n);

x1 = leastSquare(A,b);
x2 = np.linalg.lstsq(A,b, rcond=None)[0];

print("lstsq residual:",np.linalg.norm(A@x1-b));
print("numpy lstsq reiduals:",np.linalg.norm(A@x2-b));

lstsq residual: 0.46921473180502676
numpy lstsq reiduals: 0.4692147318050268


**Eigenvalues and Eigenvectors of symmetrix Matrices**

The eigenvalues and eigenvectors of a matrix **A** can be calculated using the *QR-algorithm*, which is an iterative process that also uses QR factorization.

In [0]:
def eigen(mat):
  A = np.array(mat);
  assert A.shape[0] == A.shape[1];      ## A has to be square
  assert np.allclose(A, A.transpose()); ## A has to be symmetric
  U = np.eye(A.shape[0]);

  for i in range(100):
    Q,R = qrFactorize(A);
    A = R @ Q;
    U = U @ Q;

  return A, U;



In [0]:
m = np.random.randint(2,10);

A = np.random.rand(m,m)
A = (A+A.transpose())/2;  ## Make A symmetric, otherwise we will have complex eigenvalues
T,U = eigen(A);

for i in range(len(T)):
  print(np.linalg.det(A-T[i,i]*np.eye(m)));
print();

for i in range(len(U)):
  print(np.linalg.norm(A@U[:,i] - T[i,i]*U[:,i]));

5.18973465712234e-10
1.7480786628199113e-14
-1.1368955906382902e-05
-8.92380084304716e-06
1.1383395701359398e-16
2.7969732514831746e-18
5.604445707379144e-18
7.95174644165735e-17
2.2087428892039665e-18

7.421074851107613e-15
1.1679618048569541e-14
0.003795829633934062
0.003795829633933979
1.809783439170562e-09
1.8097849461602667e-09
2.814611826769246e-15
8.359126874403208e-15
2.1024262926272527e-14


# **Results**

In [15]:
class TestMatrixMethods(unittest.TestCase):

  def test_sparse(self):
    for i in range(100):
      m = np.random.randint(2,10);
      n = np.random.randint(2,10);
      dens = np.random.uniform(low=1/(m*n), high=0.9);
      A = csr_matrix(sparse.random(m,n,dens));
      x = random.rand(n);

      self.assertTrue(np.allclose(A @ x, sparseProduct(x, A.data, A.indices, A.indptr)));


  def test_qrFactorize(self):
    for i in range(100):
      n = np.random.randint(2,10);
      m = np.random.randint(n,10);  ## m >= n for QR to work (kind of)

      A = np.random.rand(m,n);
      Q,R = qrFactorize(A);
      q,r = np.linalg.qr(A);

      self.assertTrue(np.allclose(R, np.triu(R, 0)));
      self.assertTrue(np.linalg.norm((Q @ R) - A, ord='fro') < 10**(-6));
      self.assertTrue(np.linalg.norm((Q.transpose() @ Q) - np.eye(n), ord='fro') < 10**(-6));
      self.assertTrue(np.allclose(np.abs(Q),np.abs(q)) and np.allclose(np.abs(R),np.abs(r)));

  def test_directSolver(self):
    for i in range(100):
      m = np.random.randint(2,10);

      A = np.random.rand(m,m)*1000-500;
      b = np.random.rand(m)*1000-500;
      x = solve(A,b);
      self.assertTrue(np.linalg.norm(A@x - b) < 10**(-6));

      A = np.random.rand(m,m)*1000-500;
      y = np.random.rand(m)*1000-500;
      b = A@y;
      x = solve(A,b);
      self.assertTrue(np.linalg.norm(x-y) < 10**(-6));

  def test_leastSquares(self):
    for i in range(100):
      m = np.random.randint(2,10);
      n = m + np.random.randint(2,10);

      A = np.random.rand(n,m);
      b = np.random.rand(n);

      x1 = leastSquare(A,b);
      x2 = np.linalg.lstsq(A,b, rcond=None)[0];

      self.assertTrue(np.linalg.norm(A@x1-b) - np.linalg.norm(A@x2-b) < 10**(-6));


  def test_eigen(self):
    err = 0;
    M = 0;
    for i in range(100):
      m = np.random.randint(2,10);
      M += m;

      A = np.random.rand(m,m)
      A = (A+A.transpose())/2;
      T,U = eigen(A);

      err = 0;
      for i in range(len(T)):
        if(np.linalg.det(A-T[i,i]*np.eye(m)) > 10**(-6)):
          err+=1;

      for i in range(len(U)):
        if(np.linalg.norm(A@U[:,i] - T[i,i]*U[:,i]) > 10**(-6)):
          err += 1;

    ## sometimes the convergence is kind of poor, and the eigenvalue
    ## is iffy. It's correct most of the time, but not always
    self.assertTrue(err < 2*M/50);


unittest.main(argv=[''], verbosity=2, exit=False)

test_directSolver (__main__.TestMatrixMethods) ... ok
test_eigen (__main__.TestMatrixMethods) ... ok
test_leastSquares (__main__.TestMatrixMethods) ... ok
test_qrFactorize (__main__.TestMatrixMethods) ... ok
test_sparse (__main__.TestMatrixMethods) ... ok

----------------------------------------------------------------------
Ran 5 tests in 2.212s

OK


The algorithms all perform as prescribed. 

# **Discussion**

Summarize your results and your conclusions. Were the results expected or surprising. Do your results have implications outside the particular problem investigated in this report? 

Overall, the algorithms works mostly as intended. QR factorization required $m\geq n$. For matrices where $m< n$ some modifications would need to be made to the algorithm, that would produce **R** as a not square matrix. This was not done, and not needed to get least squares and eigen value factorization to work.